# Suburb Venue Profiles and the Sydney Housing Market

### Scrape List of Sydney Suburbs from Webpage into Dataframe

In [72]:
link = "http://www.walksydneystreets.net/suburbssydneyall.htm"

In [78]:
# The code was removed by Watson Studio for sharing.

""
index
Abbotsbury
Abbotsford
Acacia Gardens
Agnes Banks
